In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
run_root_dir = '../results/dswe/pr_temp_tmin_tmax_dah_trasp_elev_slopeGTOPO_aspectGTOPO_pr_t7_temp_t7_tmin_t7_tmax_t7/'

In [ ]:
save_dir = run_root_dir+'all_k/'

In [ ]:
mode = 'val'   # 'train'

In [ ]:
nfolds = 4

In [ ]:
os.makedirs(save_dir+'train/daily_eval/', exist_ok=True)
os.makedirs(save_dir+'val/daily_eval/', exist_ok=True)
os.makedirs(save_dir+'train/cum_eval/', exist_ok=True)
os.makedirs(save_dir+'val/cum_eval/', exist_ok=True)
os.makedirs(save_dir+'train/swe_eval/', exist_ok=True)
os.makedirs(save_dir+'val/swe_eval/', exist_ok=True)

In [ ]:
def calculate_r2_mae_mse(actual, predicted):
    mse = np.sum((actual - predicted)**2)/len(actual)
    mae = np.mean(abs(predicted - actual))
    r2 = 1 - (mse/np.var(actual))
    return r2, mae, mse

In [ ]:
all_sites_df = pd.read_csv('../data/all_sites_df.csv', index_col=0)

In [ ]:
features = ['pr', 'tmin', 'tmax', 'temp', 'sd', 'swe_t1', 'dah', 'trasp', 'elev', 'elevGTOPO', 'slopeGTOPO', 'aspectGTOPO', 
            'pr_t1', 'temp_t1', 'tmin_t1', 'tmax_t1', 'pr_t2', 'temp_t2', 'tmin_t2', 'tmax_t2', 'pr_t3', 'temp_t3', 'tmin_t3', 'tmax_t3', 
            'pr_t4', 'temp_t4', 'tmin_t4', 'tmax_t4', 'pr_t5', 'temp_t5', 'tmin_t5', 'tmax_t5', 'pr_t6', 'temp_t6', 'tmin_t6', 'tmax_t6', 'pr_t7', 'temp_t7', 'tmin_t7', 'tmax_t7']

In [ ]:
nlags = 7

In [ ]:
run_daily_dfs_lst = []

In [ ]:
for k in range(nfolds):
    run_fold_daily_dir = f'{run_root_dir}k={k}/daily_eval/{mode}/{mode}_set.csv'
    run_fold_daily_df = pd.read_csv(run_fold_daily_dir, index_col=0)
    run_daily_dfs_lst.append(run_fold_daily_df)

In [ ]:
run_daily_dfs = pd.concat(run_daily_dfs_lst)

In [ ]:
run_daily_dfs = run_daily_dfs[~pd.to_datetime(run_daily_dfs['datetime']).dt.month.isin([7, 8, 9])]

##### Begin: Outliers?

In [ ]:
# run_daily_dfs = run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] > -100]
run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] < -1000]

In [ ]:
run_daily_dfs[run_daily_dfs['pred_daily_abl_P2M'] > 1000]#[['tmin_t1', 'tmin_t2', 'tmin_t3', 'tmin_t4', 'tmin_t5', 'tmin_t6', 'tmin_t7']]

In [ ]:
run_daily_dfs[run_daily_dfs['pred_daily_abl_P3M'] > 1000]#[features[:20]]

In [ ]:
run_daily_dfs[run_daily_dfs['pred_daily_abl_P3M'] < -1000]

In [ ]:
run_daily_dfs[run_daily_dfs['pred_daily_abl_P2M'] < -1000]

In [ ]:
run_daily_dfs.shape

In [ ]:
print(np.nanmin(run_daily_dfs['daily_abl']), np.nanmax(run_daily_dfs['daily_abl']))
print(np.nanmin(run_daily_dfs['pred_daily_abl_FIG2_match']), np.nanmax(run_daily_dfs['pred_daily_abl_FIG2_match']))
print(np.nanmin(run_daily_dfs['pred_daily_abl_LM']), np.nanmax(run_daily_dfs['pred_daily_abl_LM']))
print(np.nanmin(run_daily_dfs['pred_daily_abl_P2M']), np.nanmax(run_daily_dfs['pred_daily_abl_P2M']))
print(np.nanmin(run_daily_dfs['pred_daily_abl_P3M']), np.nanmax(run_daily_dfs['pred_daily_abl_P3M']))
print(np.nanmin(run_daily_dfs['pred_daily_abl_RF']), np.nanmax(run_daily_dfs['pred_daily_abl_RF']))
print(np.nanmin(run_daily_dfs['pred_daily_abl_NN']), np.nanmax(run_daily_dfs['pred_daily_abl_NN']))

In [ ]:
run_daily_dfs = run_daily_dfs[(run_daily_dfs['pred_daily_abl_P2M'] < 1000)]
run_daily_dfs = run_daily_dfs[(run_daily_dfs['pred_daily_abl_P3M'] < 1000)]
run_daily_dfs = run_daily_dfs[(run_daily_dfs['pred_daily_abl_P2M'] > -1000)]
run_daily_dfs = run_daily_dfs[(run_daily_dfs['pred_daily_abl_P3M'] > -1000)]

In [ ]:
len(run_daily_dfs)

##### End: Outliers?

### Aggregate

In [ ]:
# run_daily_dfs = run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] != -528.0009622082349]
# run_daily_dfs = run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] != -367.4994889711954]

In [ ]:
# run_daily_dfs = run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] > -1000]

In [ ]:
# np.min(run_daily_dfs['pred_daily_abl_LM'])

In [ ]:
# run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] == -528.0009622082349].index

In [ ]:
# run_daily_dfs[(run_daily_dfs['sitecode'] == '481_UT_SNTL') & (run_daily_dfs['water_year'] == 2002)]

In [ ]:
xminimum, xmaximum = min(run_daily_dfs['daily_abl']), max(run_daily_dfs['daily_abl'])
yminimum = min(np.min(run_daily_dfs['pred_daily_abl_FIG2_match']), np.min(run_daily_dfs['pred_daily_abl_LM']), np.min(run_daily_dfs['pred_daily_abl_P2M']), np.min(run_daily_dfs['pred_daily_abl_P3M']), np.min(run_daily_dfs['pred_daily_abl_RF']), np.min(run_daily_dfs['pred_daily_abl_NN'])) 
ymaximum = max(np.max(run_daily_dfs['pred_daily_abl_FIG2_match']), np.max(run_daily_dfs['pred_daily_abl_LM']), np.max(run_daily_dfs['pred_daily_abl_P2M']), np.max(run_daily_dfs['pred_daily_abl_P3M']), np.max(run_daily_dfs['pred_daily_abl_RF']), np.max(run_daily_dfs['pred_daily_abl_NN'])) 
minimum, maximum = min(xminimum, yminimum), max(xmaximum, ymaximum)

fig, axes = plt.subplots(2, 3, figsize=(18,10))
axes[0, 0].plot(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_FIG2_match'], 'bo', markersize=1, alpha=0.2) #row=0, col=0
axes[0, 1].plot(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_LM'], 'bo', markersize=1, alpha=0.2) #row=0, col=0
axes[0, 2].plot(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_P2M'], 'bo', markersize=1, alpha=0.2) #row=1, col=0
axes[1, 0].plot(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_P3M'], 'bo', markersize=1, alpha=0.2) #row=0, col=1
axes[1, 1].plot(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_RF'], 'bo', markersize=1, alpha=0.2) #row=1, col=1
axes[1, 2].plot(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_NN'], 'bo', markersize=1, alpha=0.2) #row=1, col=1
for i in range(2):
    for j in range(3):
        # if i != 1 or j != 2:
        axes[i,j].axline([0, 0], slope=1, color='grey', alpha=0.75)
        axes[i,j].axhline(0, xmin=minimum, xmax=maximum, color='red', alpha=0.5)
        axes[i,j].axvline(0, ymin=minimum, ymax=maximum, color='red', alpha=0.5)
        axes[i,j].set_xlim(minimum, maximum)
        axes[i,j].set_ylim(minimum, maximum)
        axes[i,j].set_xlabel('True Daily Ablation (mm)')
# axes[-1, -1].axis('off')
axes[0, 0].set_ylabel('UASWE Predicted Daily Ablation (mm)')
axes[0, 1].set_ylabel('LM Predicted Daily Ablation (mm)')
axes[0, 2].set_ylabel('P2M Predicted Daily Ablation (mm)')
axes[1, 0].set_ylabel('P3M Predicted Daily Ablation (mm)')
axes[1, 1].set_ylabel('RF Predicted Daily Ablation (mm)')
axes[1, 2].set_ylabel('NN Predicted Daily Ablation (mm)')
plt.savefig(f'{save_dir}{mode}/daily_eval/daily_abl_scatter.png', dpi=300)
# plt.savefig(f'{save_dir}{mode}/daily_eval/daily_abl_scatter_no_outlier.png', dpi=300)
# plt.show()

In [ ]:
# FIX THE y=x LINE AND x=0 LINE!!!
fig, axes = plt.subplots(2, 3, figsize=(30,10))
sns.boxplot(data=run_daily_dfs.round(2), x='daily_abl', y='pred_daily_abl_FIG2_match', ax=axes[0,0], flierprops={'marker': '.', 'markersize': 5})
sns.boxplot(data=run_daily_dfs.round(2), x='daily_abl', y='pred_daily_abl_LM', ax=axes[0,1], flierprops={'marker': '.', 'markersize': 5})
sns.boxplot(data=run_daily_dfs.round(2), x='daily_abl', y='pred_daily_abl_P2M', ax=axes[0,2], flierprops={'marker': '.', 'markersize': 5})
sns.boxplot(data=run_daily_dfs.round(2), x='daily_abl', y='pred_daily_abl_P3M', ax=axes[1,0], flierprops={'marker': '.', 'markersize': 5})
sns.boxplot(data=run_daily_dfs.round(2), x='daily_abl', y='pred_daily_abl_RF', ax=axes[1,1], flierprops={'marker': '.', 'markersize': 5})
sns.boxplot(data=run_daily_dfs.round(2), x='daily_abl', y='pred_daily_abl_NN', ax=axes[1,2], flierprops={'marker': '.', 'markersize': 5})
for i in range(2):
    for j in range(3):
        axes[i,j].set_xticklabels(labels=np.unique(run_daily_dfs['daily_abl'].round(2)), rotation=90)
        # axes[i,j].axline([0, 0], slope=1, color='grey', linestyle='--', alpha=0.5)
        axes[i,j].axhline(0, xmin=np.unique(run_daily_dfs['daily_abl'].round(2))[0], xmax=np.unique(run_daily_dfs['daily_abl'].round(2))[-1], color='grey', linestyle='--', alpha=0.5)
        # axes[i,j].axvline(0, ymin=minimum, ymax=maximum, color='grey', linestyle='--', alpha=0.5)
        axes[i,j].set_ylim(yminimum, ymaximum)
        # axes[i,j].set_ylim(-50, 80)
        # axes[i,j].set_xlabel('True Daily Ablation (mm)')
plt.savefig(f'{save_dir}{mode}/daily_eval/daily_abl_scatter_boxplot.png', dpi=300)
# plt.savefig(f'{save_dir}{mode}/daily_eval/daily_abl_scatter_boxplot_no_outlier.png', dpi=300)

In [ ]:
daily_UASWE_r2, daily_UASWE_mse = calculate_r2_mse(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_FIG2_match'])
daily_LM_r2, daily_LM_mse = calculate_r2_mse(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_LM'])
daily_P2M_r2, daily_P2M_mse = calculate_r2_mse(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_P2M'])
daily_P3M_r2, daily_P3M_mse = calculate_r2_mse(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_P3M'])
daily_RF_r2, daily_RF_mse = calculate_r2_mse(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_RF'])
daily_NN_r2, daily_NN_mse = calculate_r2_mse(run_daily_dfs['daily_abl'], run_daily_dfs['pred_daily_abl_NN'])

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
daily_UASWE_r2, daily_UASWE_mae, daily_UASWE_mse = calculate_r2_mse(run_daily_dfs[['daily_abl', 'pred_daily_abl_FIG2_match']].dropna()['daily_abl'], 
                                                                   run_daily_dfs[['daily_abl', 'pred_daily_abl_FIG2_match']].dropna()['pred_daily_abl_FIG2_match'])
daily_LM_r2, daily_LM_mae, daily_LM_mse = calculate_r2_mse(run_daily_dfs[['daily_abl', 'pred_daily_abl_LM']].dropna()['daily_abl'], 
                                             run_daily_dfs[['daily_abl', 'pred_daily_abl_LM']].dropna()['pred_daily_abl_LM'])
daily_P2M_r2, daily_P2M_mae, daily_P2M_mse = calculate_r2_mse(run_daily_dfs[['daily_abl', 'pred_daily_abl_P2M']].dropna()['daily_abl'], 
                                               run_daily_dfs[['daily_abl', 'pred_daily_abl_P2M']].dropna()['pred_daily_abl_P2M'])
daily_P3M_r2, daily_P3M_mae, daily_P3M_mse = calculate_r2_mse(run_daily_dfs[['daily_abl', 'pred_daily_abl_P3M']].dropna()['daily_abl'], 
                                               run_daily_dfs[['daily_abl', 'pred_daily_abl_P3M']].dropna()['pred_daily_abl_P3M'])
daily_RF_r2, daily_RF_mae, daily_RF_mse = calculate_r2_mse(run_daily_dfs[['daily_abl', 'pred_daily_abl_RF']].dropna()['daily_abl'], 
                                             run_daily_dfs[['daily_abl', 'pred_daily_abl_RF']].dropna()['pred_daily_abl_RF'])
daily_NN_r2, daily_NN_mae, daily_NN_mse = calculate_r2_mse(run_daily_dfs[['daily_abl', 'pred_daily_abl_NN']].dropna()['daily_abl'], 
                                             run_daily_dfs[['daily_abl', 'pred_daily_abl_NN']].dropna()['pred_daily_abl_NN'])

In [ ]:
calculate_r2_mse(run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] > -100][['daily_abl', 'pred_daily_abl_FIG2_match']].dropna()['daily_abl'], 
                 run_daily_dfs[run_daily_dfs['pred_daily_abl_FIG2_match'] > -100][['daily_abl', 'pred_daily_abl_FIG2_match']].dropna()['pred_daily_abl_FIG2_match'])


In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
print(daily_UASWE_r2, daily_LM_r2, daily_P2M_r2, daily_P3M_r2, daily_RF_r2, daily_NN_r2)
print(daily_UASWE_mse, daily_LM_mse, daily_P2M_mse, daily_P3M_mse, daily_RF_mse, daily_NN_mse)

In [ ]:
# np.sum() and np.mean() replace NANs with 0.0

In [ ]:
daily_metrics = {'UA SWE Reg': [daily_UASWE_mse, daily_UASWE_mae, daily_UASWE_r2],
        'Lin Reg': [daily_LM_mse, daily_LM_mae, daily_LM_r2],
        'Quad Reg': [daily_P2M_mse, daily_P2M_mae, daily_P2M_r2],
        # # 'Quad Reg w/ Int': [p2im_mse, p2im_mae, p2im_rsq],
        'Cub Reg': [daily_P3M_mse, daily_P3M_mae, daily_P3M_r2],
        'RF': [daily_RF_mse, daily_RF_mae, daily_RF_r2],
        'NN': [daily_NN_mse, daily_NN_mae, daily_NN_r2]
        # # 'GLM': [glm_mse, glm_mae, glm_rsq]
       }

In [ ]:
# Creates pandas DataFrame.
daily_metrics_df = pd.DataFrame(daily_metrics, index=['MSE', 'MAE', 'R2'])

In [ ]:
daily_metrics_df

In [ ]:
daily_metrics_df.to_csv(f'{save_dir}{mode}/daily_eval/daily_abl_metrics.csv')
# daily_metrics_df.to_csv(f'{save_dir}{mode}/daily_eval/daily_abl_metrics_no_outlier.csv')

### Climatological

In [ ]:
freqs = run_daily_dfs.groupby([(pd.to_datetime(run_daily_dfs['datetime']).dt.month),(pd.to_datetime(run_daily_dfs['datetime']).dt.day)])['pred_daily_abl_LM'].size()
freqs.index.names=['Month', 'Day']
freqs = freqs.to_frame()

In [ ]:
# create datetime column in freqs df

In [ ]:
freqs['date'] = freqs.index.get_level_values('Month').astype(str) + '-' + freqs.index.get_level_values('Day').astype(str)

In [ ]:
freqs['month'] = [str(i).zfill(2) for i in freqs.index.get_level_values('Month')]

In [ ]:
freqs['day'] = [str(i).zfill(2) for i in freqs.index.get_level_values('Day')]

In [ ]:
freqs['datetime'] = [datetime.datetime(2001, int(row['month']), int(row['day'])) for idx, row in freqs.iterrows()]

In [ ]:
# separate df of all dates in a year

In [ ]:
yr_dates = pd.date_range(start="2001-01-01",end="2001-12-31").to_frame(name='datetime')

In [ ]:
yr_dates['month'] = [str(i).zfill(2) for i in pd.to_datetime(yr_dates['datetime']).dt.month]

In [ ]:
yr_dates['day'] = [str(i).zfill(2) for i in pd.to_datetime(yr_dates['datetime']).dt.day]

In [ ]:
yr_dates['date'] = pd.to_datetime(yr_dates['datetime']).dt.month.astype(str) + '-' + pd.to_datetime(yr_dates['datetime']).dt.day.astype(str)

In [ ]:
# merge dfs

In [ ]:
full_yr = freqs.merge(yr_dates, how='outer', on=['datetime', 'month', 'day', 'date']).fillna(0, inplace = False).sort_values(by='datetime', ascending=True)

In [ ]:
full_yr = full_yr.reset_index(drop=True)

In [ ]:
# 10042023
fig, ax = plt.subplots(figsize=(24, 8))
ax.plot(pd.concat([full_yr[273:]['date'], full_yr[:181]['date']]).values, pd.concat([full_yr[273:]['pred_daily_abl_LM'], full_yr[:181]['pred_daily_abl_LM']]).values, label='SNOTEL SWE', color='black')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([full_yr[273:]['date'], full_yr[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Frequency of Site-Years in Post-Processed Validation Set')
ax.set_ylabel('Number of Site-Years')
ax.set_xlabel('Date')
# ax.axhline(5, color='grey', linestyle='--', linewidth=0.8)
# ax.axhline(2, color='grey', linestyle='--', linewidth=0.8)
# ax.axhline(1, color='grey', linestyle='--', linewidth=0.8)
ax.axhline(0, color='grey', linestyle='--', linewidth=0.8)
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
plt.savefig(f'{save_dir}{mode}/daily_eval/freq_by_date.png', dpi=300)

##### Maybe Use: Begin

In [ ]:
UASWE_metrics = run_daily_dfs[['datetime', 'daily_abl', 'pred_daily_abl_FIG2_match']].rename(columns={'daily_abl':'Actual', 'pred_daily_abl_FIG2_match':'Predicted'}).groupby([pd.to_datetime(run_daily_dfs['datetime']).dt.month, pd.to_datetime(run_daily_dfs['datetime']).dt.day]).apply(metrics)
UASWE_metrics.index.names=['Month','Day']
# UASWE_metrics.index.get_level_values('Month')
# UASWE_metrics = UASWE_metrics.drop([7,8,9], level='Month')

LM_metrics = run_daily_dfs[['datetime', 'daily_abl', 'pred_daily_abl_LM']].rename(columns={'daily_abl':'Actual', 'pred_daily_abl_LM':'Predicted'}).groupby([pd.to_datetime(run_daily_dfs['datetime']).dt.month, pd.to_datetime(run_daily_dfs['datetime']).dt.day]).apply(metrics)
LM_metrics.index.names=['Month','Day']
# LM_metrics.index.get_level_values('Month')
# LM_metrics = LM_metrics.drop([7,8,9], level='Month')

P2M_metrics = run_daily_dfs[['datetime', 'daily_abl', 'pred_daily_abl_P2M']].rename(columns={'daily_abl':'Actual', 'pred_daily_abl_P2M':'Predicted'}).groupby([pd.to_datetime(run_daily_dfs['datetime']).dt.month, pd.to_datetime(run_daily_dfs['datetime']).dt.day]).apply(metrics)
P2M_metrics.index.names=['Month','Day']
# P2M_metrics.index.get_level_values('Month')
# P2M_metrics = P2M_metrics.drop([7,8,9], level='Month')

P3M_metrics = run_daily_dfs[['datetime', 'daily_abl', 'pred_daily_abl_P3M']].rename(columns={'daily_abl':'Actual', 'pred_daily_abl_P3M':'Predicted'}).groupby([pd.to_datetime(run_daily_dfs['datetime']).dt.month, pd.to_datetime(run_daily_dfs['datetime']).dt.day]).apply(metrics)
P3M_metrics.index.names=['Month','Day']
# P3M_metrics.index.get_level_values('Month')
# P3M_metrics = P3M_metrics.drop([7,8,9], level='Month')

RF_metrics = run_daily_dfs[['datetime', 'daily_abl', 'pred_daily_abl_RF']].rename(columns={'daily_abl':'Actual', 'pred_daily_abl_RF':'Predicted'}).groupby([pd.to_datetime(run_daily_dfs['datetime']).dt.month, pd.to_datetime(run_daily_dfs['datetime']).dt.day]).apply(metrics)
RF_metrics.index.names=['Month','Day']
# RF_metrics.index.get_level_values('Month')
# RF_metrics = RF_metrics.drop([7,8,9], level='Month')

NN_metrics = run_daily_dfs[['datetime', 'daily_abl', 'pred_daily_abl_NN']].rename(columns={'daily_abl':'Actual', 'pred_daily_abl_NN':'Predicted'}).groupby([pd.to_datetime(run_daily_dfs['datetime']).dt.month, pd.to_datetime(run_daily_dfs['datetime']).dt.day]).apply(metrics)
NN_metrics.index.names=['Month','Day']
# NN_metrics.index.get_level_values('Month')
# NN_metrics = NN_metrics.drop([7,8,9], level='Month')

In [ ]:
UASWE_metrics['date'] = UASWE_metrics.index.get_level_values('Month').astype(str) + '-' + UASWE_metrics.index.get_level_values('Day').astype(str)
LM_metrics['date'] = LM_metrics.index.get_level_values('Month').astype(str) + '-' + LM_metrics.index.get_level_values('Day').astype(str)
P2M_metrics['date'] = P2M_metrics.index.get_level_values('Month').astype(str) + '-' + P2M_metrics.index.get_level_values('Day').astype(str)
P3M_metrics['date'] = P3M_metrics.index.get_level_values('Month').astype(str) + '-' + P3M_metrics.index.get_level_values('Day').astype(str)
RF_metrics['date'] = RF_metrics.index.get_level_values('Month').astype(str) + '-' + RF_metrics.index.get_level_values('Day').astype(str)
NN_metrics['date'] = NN_metrics.index.get_level_values('Month').astype(str) + '-' + NN_metrics.index.get_level_values('Day').astype(str)

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mse'], UASWE_metrics[:181]['mse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mse'], LM_metrics[:181]['mse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mse'], P2M_metrics[:181]['mse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mse'], P3M_metrics[:181]['mse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mse'], RF_metrics[:181]['mse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mse'], NN_metrics[:181]['mse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: MSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_pth}cum_eval/climatological_scheme/{mode}/cum_abl_mse_plot.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['rmse'], UASWE_metrics[:181]['rmse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['rmse'], LM_metrics[:181]['rmse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['rmse'], P2M_metrics[:181]['rmse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['rmse'], P3M_metrics[:181]['rmse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['rmse'], RF_metrics[:181]['rmse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['rmse'], NN_metrics[:181]['rmse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: RMSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_pth}cum_eval/climatological_scheme/{mode}/cum_abl_mse_plot.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mae'], UASWE_metrics[:181]['mae']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mae'], LM_metrics[:181]['mae']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mae'], P2M_metrics[:181]['mae']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mae'], P3M_metrics[:181]['mae']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mae'], RF_metrics[:181]['mae']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mae'], NN_metrics[:181]['mae']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: MAE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_pth}cum_eval/climatological_scheme/{mode}/cum_abl_mae_plot.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['r2'], UASWE_metrics[:181]['r2']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['r2'], LM_metrics[:181]['r2']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['r2'], P2M_metrics[:181]['r2']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['r2'], P3M_metrics[:181]['r2']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['r2'], RF_metrics[:181]['r2']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['r2'], NN_metrics[:181]['r2']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: R2')
ax.legend()
# ax.set_ylim(-1, 1)
ax.axhline(0, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_pth}cum_eval/climatological_scheme/{mode}/cum_abl_r2_plot.png', dpi=300)
plt.show()

##### Maybe Use: End

In [ ]:
run_cum_dfs_lst = []

In [ ]:
for k in range(nfolds):
    exclude_lst = []
    with open(f"/global/cfs/cdirs/m1517/yum/swe/UofA_improvements/regression/modeling_12122023/k{k}_site_years_exclude.txt", "r") as f:
    # with open(f"/global/cfs/cdirs/m1517/yum/swe/UofA_improvements/regression/modeling_12122023/k{k}_site_years_exclude_SDSUB.txt", "r") as f:
        for line in f:
            exclude_lst.append(line.strip())
            
    k_run_cum_dfs_lst = []
    # print(k)
            
    run_fold_cum_dir = f'{run_root_dir}k={k}/cum_swe_ts/{mode}'
    for file in os.listdir(run_fold_cum_dir):
        filename = os.fsdecode(file)
        if filename.endswith("_df.csv") and filename not in exclude_lst: 
            # print(os.path.join(directory, filename))
            # print(filename)
            df = pd.read_csv(os.path.join(run_fold_cum_dir, filename), index_col=0)
            k_run_cum_dfs_lst.append(df)
        # else:
        #     print(filename)
    run_fold_cum_df = pd.concat(k_run_cum_dfs_lst)
    
    run_cum_dfs_lst.append(run_fold_cum_df)

In [ ]:
run_cum_dfs = pd.concat(run_cum_dfs_lst)

In [ ]:
run_cum_dfs = run_cum_dfs[~pd.to_datetime(run_cum_dfs['datetime']).dt.month.isin([7, 8, 9])]

#### Begin: Full, no exclude

In [ ]:
run_cum_dfs_lst = []

In [ ]:
for k in range(nfolds):
    run_fold_cum_dir = f'{run_root_dir}k={k}/cum_swe_ts/{mode}/ALL_SITES_WY_DF.csv'
    run_fold_cum_df = pd.read_csv(run_fold_cum_dir, index_col=0)
    run_cum_dfs_lst.append(run_fold_cum_df)

In [ ]:
run_cum_dfs = pd.concat(run_cum_dfs_lst)

In [ ]:
run_cum_dfs.keys()

In [ ]:
run_cum_dfs[(run_cum_dfs['temp_t7'].isna())][features][:60]

In [ ]:
run_cum_dfs[['temp_t1', 'temp_t2', 'temp_t3', 'temp_t4', 'temp_t5', 'temp_t6', 'temp_t7']].loc[345165:345170]

In [ ]:
run_cum_dfs[run_cum_dfs['temp_t7'].isna()][['datetime','sitecode','pred_daily_acc_abl_P3M', 'pred_acc_abl_P3M', 'pred_acc_abl_RF', 'pred_daily_acc_abl_RF', 'pred_swe_P3M','pred_swe_RF']][:60]

In [ ]:
run_cum_dfs.dropna(subset=features, how='any')

In [ ]:
run_cum_dfs[features]

In [ ]:
run_cum_dfs

In [ ]:
run_cum_dfs = run_cum_dfs[~pd.to_datetime(run_cum_dfs['datetime']).dt.month.isin([7, 8, 9])]

#### End: Full, no exclude

### Aggregate

In [ ]:
xminimum, xmaximum = min(run_cum_dfs['abl_snw']), max(run_cum_dfs['abl_snw'])
yminimum = min(np.min(run_cum_dfs['pred_acc_abl_UASWE']), np.min(run_cum_dfs['pred_acc_abl_LM']), np.min(run_cum_dfs['pred_acc_abl_P2M']), np.min(run_cum_dfs['pred_acc_abl_P3M']), np.min(run_cum_dfs['pred_acc_abl_RF']), np.min(run_cum_dfs['pred_acc_abl_NN'])) 
ymaximum = max(np.max(run_cum_dfs['pred_acc_abl_UASWE']), np.max(run_cum_dfs['pred_acc_abl_LM']), np.max(run_cum_dfs['pred_acc_abl_P2M']), np.max(run_cum_dfs['pred_acc_abl_P3M']), np.max(run_cum_dfs['pred_acc_abl_RF']), np.max(run_cum_dfs['pred_acc_abl_NN'])) 
minimum, maximum = min(xminimum, yminimum), max(xmaximum, ymaximum)

fig, axes = plt.subplots(2, 3, figsize=(18,10))
col = np.where(run_cum_dfs['pred_acc_snw_UASWE'] == run_cum_dfs['pred_acc_abl_UASWE'], 'r', 'b')
axes[0, 0].scatter(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_UASWE'], s=1, alpha=0.05, c=col)  #, 'bo', markersize=1, alpha=0.1) #row=0, col=0
axes[0, 1].scatter(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_LM'], s=1, alpha=0.05, c=col)  #, 'bo', markersize=1, alpha=0.1) #row=0, col=0
axes[0, 2].scatter(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_P2M'], s=1, alpha=0.05, c=col)  #, 'bo', markersize=1, alpha=0.1) #row=1, col=0
axes[1, 0].scatter(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_P3M'], s=1, alpha=0.05, c=col)  #, 'bo', markersize=1, alpha=0.1) #row=0, col=1
axes[1, 1].scatter(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_RF'], s=1, alpha=0.05, c=col)  #, 'bo', markersize=1, alpha=0.1) #row=1, col=1
axes[1, 2].scatter(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_NN'], s=1, alpha=0.05, c=col)  #, 'bo', markersize=1, alpha=0.1) #row=1, col=1
for i in range(2):
    for j in range(3):
        # if i != 1 or j != 2:
        axes[i,j].axline([0, 0], slope=1, color='grey', alpha=0.75)
        axes[i,j].axhline(0, xmin=minimum, xmax=maximum, color='red', alpha=0.5)
        axes[i,j].axvline(0, ymin=minimum, ymax=maximum, color='red', alpha=0.5)
        axes[i,j].set_xlim(minimum, maximum)
        axes[i,j].set_ylim(minimum, maximum)
        axes[i,j].set_xlabel('True Cumulative Ablation (mm)')
# axes[-1, -1].axis('off')
axes[0, 0].set_ylabel('UASWE Predicted Cumulative Ablation (mm)')
axes[0, 1].set_ylabel('LM Predicted Cumulative Ablation (mm)')
axes[0, 2].set_ylabel('P2M Predicted Cumulative Ablation (mm)')
axes[1, 0].set_ylabel('P3M Predicted Cumulative Ablation (mm)')
axes[1, 1].set_ylabel('RF Predicted Cumulative Ablation (mm)')
axes[1, 2].set_ylabel('NN Predicted Cumulative Ablation (mm)')
plt.savefig(f'{save_dir}{mode}/cum_eval/cum_abl_scatter.png', dpi=300)
# plt.show()

In [ ]:
xminimum, xmaximum = min(run_cum_dfs['abl_snw']), max(run_cum_dfs['abl_snw'])
yminimum = min(np.min(run_cum_dfs['pred_acc_abl_UASWE']), np.min(run_cum_dfs['pred_acc_abl_LM']), np.min(run_cum_dfs['pred_acc_abl_P2M']), np.min(run_cum_dfs['pred_acc_abl_P3M']), np.min(run_cum_dfs['pred_acc_abl_RF']), np.min(run_cum_dfs['pred_acc_abl_NN'])) 
ymaximum = max(np.max(run_cum_dfs['pred_acc_abl_UASWE']), np.max(run_cum_dfs['pred_acc_abl_LM']), np.max(run_cum_dfs['pred_acc_abl_P2M']), np.max(run_cum_dfs['pred_acc_abl_P3M']), np.max(run_cum_dfs['pred_acc_abl_RF']), np.max(run_cum_dfs['pred_acc_abl_NN'])) 
minimum, maximum = min(xminimum, yminimum), max(xmaximum, ymaximum)

fig, axes = plt.subplots(2, 3, figsize=(18,10))
axes[0, 0].hexbin(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_UASWE'], gridsize=200, cmap='Blues', bins='log') #row=0, col=0
axes[0, 1].hexbin(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_LM'], gridsize=200, cmap='Blues', bins='log') #row=0, col=0
axes[0, 2].hexbin(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_P2M'], gridsize=200, cmap='Blues', bins='log') #row=1, col=0
axes[1, 0].hexbin(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_P3M'], gridsize=200, cmap='Blues', bins='log') #row=0, col=1
axes[1, 1].hexbin(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_RF'], gridsize=200, cmap='Blues', bins='log') #row=1, col=1
axes[1, 2].hexbin(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_NN'], gridsize=200, cmap='Blues', bins='log') #row=1, col=1
for i in range(2):
    for j in range(3):
        # if i != 1 or j != 2:
        axes[i,j].axline([0, 0], slope=1, color='grey', alpha=0.75)
        axes[i,j].axhline(0, xmin=minimum, xmax=maximum, color='red', alpha=0.5)
        axes[i,j].axvline(0, ymin=minimum, ymax=maximum, color='red', alpha=0.5)
        axes[i,j].set_xlim(minimum, maximum)
        axes[i,j].set_ylim(minimum, maximum)
        axes[i,j].set_xlabel('True Cumulative Ablation (mm)')
# axes[-1, -1].axis('off')
axes[0, 0].set_ylabel('UASWE Predicted Cumulative Ablation (mm)')
axes[0, 1].set_ylabel('LM Predicted Cumulative Ablation (mm)')
axes[0, 2].set_ylabel('P2M Predicted Cumulative Ablation (mm)')
axes[1, 0].set_ylabel('P3M Predicted Cumulative Ablation (mm)')
axes[1, 1].set_ylabel('RF Predicted Cumulative Ablation (mm)')
axes[1, 2].set_ylabel('NN Predicted Cumulative Ablation (mm)')
plt.savefig(f'{save_dir}{mode}/cum_eval/cum_abl_hexplot.png', dpi=300)
# plt.show()

In [ ]:
cum_UASWE_r2, cum_UASWE_mse = calculate_r2_mse(run_cum_dfs[['abl_snw', 'pred_acc_abl_UASWE']].dropna()['abl_snw'], 
                                                   run_cum_dfs[['abl_snw', 'pred_acc_abl_UASWE']].dropna()['pred_acc_abl_UASWE'])
cum_LM_r2, cum_LM_mse = calculate_r2_mse(run_cum_dfs[['abl_snw', 'pred_acc_abl_LM']].dropna()['abl_snw'], 
                                             run_cum_dfs[['abl_snw', 'pred_acc_abl_LM']].dropna()['pred_acc_abl_LM'])
cum_P2M_r2, cum_P2M_mse = calculate_r2_mse(run_cum_dfs[['abl_snw', 'pred_acc_abl_P2M']].dropna()['abl_snw'], 
                                               run_cum_dfs[['abl_snw', 'pred_acc_abl_P2M']].dropna()['pred_acc_abl_P2M'])
cum_P3M_r2, cum_P3M_mse = calculate_r2_mse(run_cum_dfs[['abl_snw', 'pred_acc_abl_P3M']].dropna()['abl_snw'], 
                                               run_cum_dfs[['abl_snw', 'pred_acc_abl_P3M']].dropna()['pred_acc_abl_P3M'])
cum_RF_r2, cum_RF_mse = calculate_r2_mse(run_cum_dfs[['abl_snw', 'pred_acc_abl_RF']].dropna()['abl_snw'], 
                                             run_cum_dfs[['abl_snw', 'pred_acc_abl_RF']].dropna()['pred_acc_abl_RF'])
cum_NN_r2, cum_NN_mse = calculate_r2_mse(run_cum_dfs[['abl_snw', 'pred_acc_abl_NN']].dropna()['abl_snw'], 
                                             run_cum_dfs[['abl_snw', 'pred_acc_abl_NN']].dropna()['pred_acc_abl_NN'])

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
len(run_cum_dfs)

In [ ]:
run_cum_dfs = run_cum_dfs.dropna(subset=['abl_snw', 'pred_acc_abl_UASWE', 
                                   'pred_acc_abl_LM', 'pred_acc_abl_P2M', 'pred_acc_abl_P3M', 
                                   'pred_acc_abl_RF','pred_acc_abl_NN'
                                  ], how='any', inplace=False)

In [ ]:
np.unique(run_cum_dfs[run_cum_dfs['pred_acc_abl_P2M'] < -1000][['sitecode', 'datetime', 'pr', 'temp', 'pred_acc_abl_P2M', 'pred_swe_P2M']]['sitecode'], return_counts=True)

In [ ]:
np.unique(run_cum_dfs[run_cum_dfs['pred_acc_abl_P2M'] < -10000][['sitecode', 'datetime', 'pr', 'temp', 'pred_acc_abl_P2M', 'pred_swe_P2M']]['sitecode'], return_counts=True)

In [ ]:
run_cum_dfs[np.round(run_cum_dfs['pred_swe_P2M'],1) > 2000][['sitecode', 'datetime', 'swe', 'pr', 'temp', 'pred_acc_abl_P2M', 'pred_swe_P2M']].sort_values('pred_swe_P2M')

In [ ]:
# Phys dAbl with SWE model
run_cum_dfs['pred_acc_abl_P2M'] = run_cum_dfs['pred_acc_abl_P2M'].clip(lower=-7.284402e+02)
run_cum_dfs['pred_swe_P2M'] = run_cum_dfs['pred_swe_P2M'].clip(upper=2.003520e+03)

In [ ]:
# Full dAbl with SWE model
run_cum_dfs['pred_acc_abl_P2M'] = run_cum_dfs['pred_acc_abl_P2M'].clip(lower=-3.460402e+02)
run_cum_dfs['pred_swe_P2M'] = run_cum_dfs['pred_swe_P2M'].clip(upper=2.007200e+03)

In [ ]:
# Full dAbl with SD model
run_cum_dfs['pred_acc_abl_P2M'] = run_cum_dfs['pred_acc_abl_P2M'].clip(lower=-8.173709e+02)
run_cum_dfs['pred_swe_P2M'] = run_cum_dfs['pred_swe_P2M'].clip(upper=2.003551e+03)

In [ ]:
cum_UASWE_r2, cum_UASWE_mae, cum_UASWE_mse = calculate_r2_mse(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_UASWE'])
cum_LM_r2, cum_LM_mae, cum_LM_mse = calculate_r2_mse(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_LM'])
cum_P2M_r2, cum_P2M_mae, cum_P2M_mse = calculate_r2_mse(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_P2M'])
cum_P3M_r2, cum_P3M_mae, cum_P3M_mse = calculate_r2_mse(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_P3M'])
cum_RF_r2, cum_RF_mae, cum_RF_mse = calculate_r2_mse(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_RF'])
cum_NN_r2, cum_NN_mae, cum_NN_mse = calculate_r2_mse(run_cum_dfs['abl_snw'], run_cum_dfs['pred_acc_abl_NN'])

In [ ]:
print(np.min(run_cum_dfs['pred_acc_abl_UASWE']), np.max(run_cum_dfs['pred_acc_abl_UASWE']))
print(np.min(run_cum_dfs['pred_acc_abl_LM']), np.max(run_cum_dfs['pred_acc_abl_LM']))
print(np.min(run_cum_dfs['pred_acc_abl_P2M']), np.max(run_cum_dfs['pred_acc_abl_P2M']))
print(np.min(run_cum_dfs['pred_acc_abl_P3M']), np.max(run_cum_dfs['pred_acc_abl_P3M']))
print(np.min(run_cum_dfs['pred_acc_abl_RF']), np.max(run_cum_dfs['pred_acc_abl_RF']))
print(np.min(run_cum_dfs['pred_acc_abl_NN']), np.max(run_cum_dfs['pred_acc_abl_NN']))

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
# also uaswelike with swe (attempt 2, to try to diagnose nn low r2/high mse)
print(cum_UASWE_r2, cum_LM_r2, cum_P2M_r2, cum_P3M_r2, cum_RF_r2, cum_NN_r2)
print(cum_UASWE_mse, cum_LM_mse, cum_P2M_mse, cum_P3M_mse, cum_RF_mse, cum_NN_mse)

In [ ]:
len(run_cum_dfs)

In [ ]:
cum_metrics = {'UA SWE Reg': [cum_UASWE_mse, cum_UASWE_mae, cum_UASWE_r2],
        'Lin Reg': [cum_LM_mse, cum_LM_mae, cum_LM_r2],
        'Quad Reg': [cum_P2M_mse, cum_P2M_mae, cum_P2M_r2],
        # # 'Quad Reg w/ Int': [p2im_mse, p2im_mae, p2im_rsq],
        'Cub Reg': [cum_P3M_mse, cum_P3M_mae, cum_P3M_r2],
        'RF': [cum_RF_mse, cum_RF_mae, cum_RF_r2],
        'NN': [cum_NN_mse, cum_NN_mae, cum_NN_r2]
        # # 'GLM': [glm_mse, glm_mae, glm_rsq]
       }

In [ ]:
# Creates pandas DataFrame.
cum_metrics_df = pd.DataFrame(cum_metrics, index=['MSE', 'MAE', 'R2'])

In [ ]:
cum_metrics_df

In [ ]:
cum_metrics_df.to_csv(f'{save_dir}{mode}/cum_eval/cum_abl_metrics.csv')

##### To delete: Begin

In [ ]:
int_df = pd.merge(run_cum_dfs, 
                  run_cum_dfs.dropna(subset=['abl_snw', 'pred_acc_abl_UASWE', 
                                   'pred_acc_abl_LM', 'pred_acc_abl_P2M', 'pred_acc_abl_P3M', 
                                   'pred_acc_abl_RF','pred_acc_abl_NN'
                                  ], how='any', inplace=False), how='left', indicator=True) 

In [ ]:
int_df[int_df['_merge']=='left_only'][['sitecode','swe','datetime','abl_snw', 'pred_acc_abl_UASWE', 
                                   'pred_acc_abl_LM', 'pred_acc_abl_P2M', 'pred_acc_abl_P3M', 
                                   'pred_acc_abl_RF','pred_acc_abl_NN'
                                  ]]

In [ ]:
calculate_r2_mse(int_df[int_df['_merge']=='left_only']['abl_snw'], int_df[int_df['_merge']=='left_only']['pred_acc_abl_UASWE'])

##### To delete: End

### Climatological

In [ ]:
UASWE_metrics = run_cum_dfs[['datetime', 'abl_snw', 'pred_acc_abl_UASWE']].rename(columns={'abl_snw':'Actual', 'pred_acc_abl_UASWE':'Predicted'}).groupby([pd.to_datetime(run_cum_dfs['datetime']).dt.month, pd.to_datetime(run_cum_dfs['datetime']).dt.day]).apply(metrics)
UASWE_metrics.index.names=['Month','Day']
# UASWE_metrics.index.get_level_values('Month')
# UASWE_metrics = UASWE_metrics.drop([7,8,9], level='Month')

LM_metrics = run_cum_dfs[['datetime', 'abl_snw', 'pred_acc_abl_LM']].rename(columns={'abl_snw':'Actual', 'pred_acc_abl_LM':'Predicted'}).groupby([pd.to_datetime(run_cum_dfs['datetime']).dt.month, pd.to_datetime(run_cum_dfs['datetime']).dt.day]).apply(metrics)
LM_metrics.index.names=['Month','Day']
# LM_metrics.index.get_level_values('Month')
# LM_metrics = LM_metrics.drop([7,8,9], level='Month')

P2M_metrics = run_cum_dfs[['datetime', 'abl_snw', 'pred_acc_abl_P2M']].rename(columns={'abl_snw':'Actual', 'pred_acc_abl_P2M':'Predicted'}).groupby([pd.to_datetime(run_cum_dfs['datetime']).dt.month, pd.to_datetime(run_cum_dfs['datetime']).dt.day]).apply(metrics)
P2M_metrics.index.names=['Month','Day']
# P2M_metrics.index.get_level_values('Month')
# P2M_metrics = P2M_metrics.drop([7,8,9], level='Month')

P3M_metrics = run_cum_dfs[['datetime', 'abl_snw', 'pred_acc_abl_P3M']].rename(columns={'abl_snw':'Actual', 'pred_acc_abl_P3M':'Predicted'}).groupby([pd.to_datetime(run_cum_dfs['datetime']).dt.month, pd.to_datetime(run_cum_dfs['datetime']).dt.day]).apply(metrics)
P3M_metrics.index.names=['Month','Day']
# P3M_metrics.index.get_level_values('Month')
# P3M_metrics = P3M_metrics.drop([7,8,9], level='Month')

RF_metrics = run_cum_dfs[['datetime', 'abl_snw', 'pred_acc_abl_RF']].rename(columns={'abl_snw':'Actual', 'pred_acc_abl_RF':'Predicted'}).groupby([pd.to_datetime(run_cum_dfs['datetime']).dt.month, pd.to_datetime(run_cum_dfs['datetime']).dt.day]).apply(metrics)
RF_metrics.index.names=['Month','Day']
# RF_metrics.index.get_level_values('Month')
# RF_metrics = RF_metrics.drop([7,8,9], level='Month')

NN_metrics = run_cum_dfs[['datetime', 'abl_snw', 'pred_acc_abl_NN']].rename(columns={'abl_snw':'Actual', 'pred_acc_abl_NN':'Predicted'}).groupby([pd.to_datetime(run_cum_dfs['datetime']).dt.month, pd.to_datetime(run_cum_dfs['datetime']).dt.day]).apply(metrics)
NN_metrics.index.names=['Month','Day']
# NN_metrics.index.get_level_values('Month')
# NN_metrics = NN_metrics.drop([7,8,9], level='Month')

In [ ]:
UASWE_metrics['date'] = UASWE_metrics.index.get_level_values('Month').astype(str) + '-' + UASWE_metrics.index.get_level_values('Day').astype(str)
LM_metrics['date'] = LM_metrics.index.get_level_values('Month').astype(str) + '-' + LM_metrics.index.get_level_values('Day').astype(str)
P2M_metrics['date'] = P2M_metrics.index.get_level_values('Month').astype(str) + '-' + P2M_metrics.index.get_level_values('Day').astype(str)
P3M_metrics['date'] = P3M_metrics.index.get_level_values('Month').astype(str) + '-' + P3M_metrics.index.get_level_values('Day').astype(str)
RF_metrics['date'] = RF_metrics.index.get_level_values('Month').astype(str) + '-' + RF_metrics.index.get_level_values('Day').astype(str)
NN_metrics['date'] = NN_metrics.index.get_level_values('Month').astype(str) + '-' + NN_metrics.index.get_level_values('Day').astype(str)

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mse'], UASWE_metrics[:181]['mse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mse'], LM_metrics[:181]['mse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mse'], P2M_metrics[:181]['mse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mse'], P3M_metrics[:181]['mse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mse'], RF_metrics[:181]['mse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mse'], NN_metrics[:181]['mse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: MSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
plt.savefig(f'{save_dir}{mode}/cum_eval/cum_abl_mse_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['rmse'], UASWE_metrics[:181]['rmse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['rmse'], LM_metrics[:181]['rmse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['rmse'], P2M_metrics[:181]['rmse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['rmse'], P3M_metrics[:181]['rmse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['rmse'], RF_metrics[:181]['rmse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['rmse'], NN_metrics[:181]['rmse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: RMSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
plt.savefig(f'{save_dir}{mode}/cum_eval/cum_abl_rmse_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mae'], UASWE_metrics[:181]['mae']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mae'], LM_metrics[:181]['mae']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mae'], P2M_metrics[:181]['mae']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mae'], P3M_metrics[:181]['mae']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mae'], RF_metrics[:181]['mae']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mae'], NN_metrics[:181]['mae']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: MAE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
plt.savefig(f'{save_dir}{mode}/cum_eval/cum_abl_mae_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['r2'], UASWE_metrics[:181]['r2']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['r2'], LM_metrics[:181]['r2']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['r2'], P2M_metrics[:181]['r2']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['r2'], P3M_metrics[:181]['r2']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['r2'], RF_metrics[:181]['r2']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['r2'], NN_metrics[:181]['r2']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Cumulative Ablation: R2')
ax.legend()
# ax.set_ylim(-1, 1)
ax.axhline(0, linestyle='--', color='lightgrey')
plt.savefig(f'{save_dir}{mode}/cum_eval/cum_abl_r2_climat.png', dpi=300)
plt.show()

### Site-Year

In [ ]:
mse_acc_abl_UASWE, mse_acc_abl_LM, mse_acc_abl_P2M, mse_acc_abl_P3M, mse_acc_abl_RF, mse_acc_abl_NN = [], [], [], [], [], []
mae_acc_abl_UASWE, mae_acc_abl_LM, mae_acc_abl_P2M, mae_acc_abl_P3M, mae_acc_abl_RF, mae_acc_abl_NN = [], [], [], [], [], []
r2_acc_abl_UASWE, r2_acc_abl_LM, r2_acc_abl_P2M, r2_acc_abl_P3M, r2_acc_abl_RF, r2_acc_abl_NN = [], [], [], [], [], []

In [ ]:
for site in np.unique(run_cum_dfs['sitecode']):
    for wy in np.unique(run_cum_dfs['water_year']):
        site_wy_df = run_cum_dfs[(run_cum_dfs['sitecode'] == site) & (run_cum_dfs['water_year'] == wy)]
        
        if not site_wy_df.empty:
            # site_wy_df_clean = site_wy_df.dropna(subset=['abl_snw', 'pred_acc_abl_UASWE', 
            #                                  'pred_acc_abl_LM', 'pred_acc_abl_P2M', 'pred_acc_abl_P3M', 
            #                                  'pred_acc_abl_RF','pred_acc_abl_NN'
            #                                 ], how='any', inplace=False)
            # site_wy_df_clean = site_wy_df_clean[~pd.to_datetime(site_wy_df_clean['datetime']).dt.month.isin([7, 8, 9])]    # evaluate only on snow season
            
            mse_acc_abl_UASWE.append(mean_squared_error(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_UASWE'], squared=True))
            mse_acc_abl_LM.append(mean_squared_error(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_LM'], squared=True))
            mse_acc_abl_P2M.append(mean_squared_error(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_P2M'], squared=True))
            mse_acc_abl_P3M.append(mean_squared_error(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_P3M'], squared=True))
            mse_acc_abl_RF.append(mean_squared_error(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_RF'], squared=True))
            mse_acc_abl_NN.append(mean_squared_error(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_NN'], squared=True))                    
            mae_acc_abl_UASWE.append(np.mean(abs(site_wy_df['pred_acc_abl_UASWE'] - site_wy_df['abl_snw'])))
            mae_acc_abl_LM.append(np.mean(abs(site_wy_df['pred_acc_abl_LM'] - site_wy_df['abl_snw'])))
            mae_acc_abl_P2M.append(np.mean(abs(site_wy_df['pred_acc_abl_P2M'] - site_wy_df['abl_snw'])))
            mae_acc_abl_P3M.append(np.mean(abs(site_wy_df['pred_acc_abl_P3M'] - site_wy_df['abl_snw'])))
            mae_acc_abl_RF.append(np.mean(abs(site_wy_df['pred_acc_abl_RF'] - site_wy_df['abl_snw'])))
            mae_acc_abl_NN.append(np.mean(abs(site_wy_df['pred_acc_abl_NN'] - site_wy_df['abl_snw'])))
            r2_acc_abl_UASWE.append(r2_score(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_UASWE']))
            r2_acc_abl_LM.append(r2_score(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_LM']))
            r2_acc_abl_P2M.append(r2_score(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_P2M']))
            r2_acc_abl_P3M.append(r2_score(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_P3M']))
            r2_acc_abl_RF.append(r2_score(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_RF']))
            r2_acc_abl_NN.append(r2_score(site_wy_df['abl_snw'], site_wy_df['pred_acc_abl_NN']))


In [ ]:
cum_metrics_med = {'UA SWE Reg': [np.median(mse_acc_abl_UASWE), np.median(mae_acc_abl_UASWE), np.median(r2_acc_abl_UASWE)],
                   'Lin Reg': [np.median(mse_acc_abl_LM), np.median(mae_acc_abl_LM), np.median(r2_acc_abl_LM)],
                   'Quad Reg': [np.median(mse_acc_abl_P2M), np.median(mae_acc_abl_P2M), np.median(r2_acc_abl_P2M)],
                   # 'Quad Reg w/ Int': [p2im_mse, p2im_mae, p2im_rsq],
                   'Cub Reg': [np.median(mse_acc_abl_P3M), np.median(mae_acc_abl_P3M), np.median(r2_acc_abl_P3M)],
                   'RF': [np.median(mse_acc_abl_RF), np.median(mae_acc_abl_RF), np.median(r2_acc_abl_RF)],
                   'NN': [np.median(mse_acc_abl_NN), np.median(mae_acc_abl_NN), np.median(r2_acc_abl_NN)]
                   # 'GLM': [glm_mse, glm_mae, glm_rsq]
                  }
cum_metrics_mean = {'UA SWE Reg': [np.mean(mse_acc_abl_UASWE), np.mean(mae_acc_abl_UASWE), np.mean(r2_acc_abl_UASWE)],
               'Lin Reg': [np.mean(mse_acc_abl_LM), np.mean(mae_acc_abl_LM), np.mean(r2_acc_abl_LM)],
               'Quad Reg': [np.mean(mse_acc_abl_P2M), np.mean(mae_acc_abl_P2M), np.mean(r2_acc_abl_P2M)],
               # 'Quad Reg w/ Int': [p2im_mse, p2im_mae, p2im_rsq],
               'Cub Reg': [np.mean(mse_acc_abl_P3M), np.mean(mae_acc_abl_P3M), np.mean(r2_acc_abl_P3M)],
               'RF': [np.mean(mse_acc_abl_RF), np.mean(mae_acc_abl_RF), np.mean(r2_acc_abl_RF)],
               'NN': [np.mean(mse_acc_abl_NN), np.mean(mae_acc_abl_NN), np.mean(r2_acc_abl_NN)]
               # 'GLM': [glm_mse, glm_mae, glm_rsq]
              }
cum_metrics_std = {'UA SWE Reg': [np.std(mse_acc_abl_UASWE), np.std(mae_acc_abl_UASWE), np.std(r2_acc_abl_UASWE)],
               'Lin Reg': [np.std(mse_acc_abl_LM), np.std(mae_acc_abl_LM), np.std(r2_acc_abl_LM)],
               'Quad Reg': [np.std(mse_acc_abl_P2M), np.std(mae_acc_abl_P2M), np.std(r2_acc_abl_P2M)],
               # 'Quad Reg w/ Int': [p2im_mse, p2im_mae, p2im_rsq],
               'Cub Reg': [np.std(mse_acc_abl_P3M), np.std(mae_acc_abl_P3M), np.std(r2_acc_abl_P3M)],
               'RF': [np.std(mse_acc_abl_RF), np.std(mae_acc_abl_RF), np.std(r2_acc_abl_RF)],
               'NN': [np.std(mse_acc_abl_NN), np.std(mae_acc_abl_NN), np.std(r2_acc_abl_NN)]
               # 'GLM': [glm_mse, glm_mae, glm_rsq]
              }

In [ ]:
cum_abl_means = pd.DataFrame(cum_metrics_med, index=['MSE','MAE','R2'])

In [ ]:
cum_abl_meds = pd.DataFrame(cum_metrics_mean, index=['MSE','MAE','R2'])

In [ ]:
cum_abl_stds = pd.DataFrame(cum_metrics_std, index=['MSE','MAE','R2'])

In [ ]:
cum_abl_means.to_csv(f'{save_dir}{mode}/cum_eval/cum_abl_means.csv')
cum_abl_meds.to_csv(f'{save_dir}{mode}/cum_eval/cum_abl_meds.csv')
cum_abl_stds.to_csv(f'{save_dir}{mode}/cum_eval/cum_abl_stds.csv')

In [ ]:
cum_abl_means

In [ ]:
cum_abl_meds

# SWE Evaluation

Load dataframes including model predictions across folds and remove summer months

In [ ]:
run_swe_dfs_lst = []

In [ ]:
for k in range(nfolds):
    run_fold_cum_dir = f'{run_root_dir}k={k}/{mode}/ALL_SITES_WY_DF.csv'
    run_fold_cum_df = pd.read_csv(run_fold_cum_dir, index_col=0)
    run_swe_dfs_lst.append(run_fold_cum_df)

In [ ]:
run_swe_dfs = pd.concat(run_swe_dfs_lst)

In [ ]:
run_swe_dfs = run_swe_dfs[~pd.to_datetime(run_swe_dfs['datetime']).dt.month.isin([7, 8, 9])]

## Aggregate

Compute performance metrics based on aggregating across all sites and dates.

In [ ]:
swe_UASWE_r2, swe_UASWE_mae, swe_UASWE_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['UASWE'])
swe_LM_r2, swe_LM_mae, swe_LM_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_LM'])
swe_P2M_r2, swe_P2M_mae, swe_P2M_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_P2M'])
swe_P3M_r2, swe_P3M_mae, swe_P3M_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_P3M'])
swe_RF_r2, swe_RF_mae, swe_RF_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_RF'])
swe_NN_r2, swe_NN_mae, swe_NN_mse = calculate_r2_mae_mse(run_swe_dfs['swe'], run_swe_dfs['pred_swe_NN'])

In [ ]:
swe_metrics = {'UA SWE Reg': [swe_UASWE_mse, swe_UASWE_mae, swe_UASWE_r2],
        'Lin Reg': [swe_LM_mse, swe_LM_mae, swe_LM_r2],
        'Quad Reg': [swe_P2M_mse, swe_P2M_mae, swe_P2M_r2],
        'Cub Reg': [swe_P3M_mse, swe_P3M_mae, swe_P3M_r2],
        'RF': [swe_RF_mse, swe_RF_mae, swe_RF_r2],
        'NN': [swe_NN_mse, swe_NN_mae, swe_NN_r2]
       }

In [ ]:
# Creates pandas DataFrame.
swe_metrics_df = pd.DataFrame(swe_metrics, index=['MSE', 'MAE', 'R2'])

In [ ]:
swe_metrics_df

In [ ]:
swe_metrics_df.to_csv(f'{save_dir}{mode}/swe_eval/aggregate_scheme/swe_abl_metrics.csv')

## Climatological

Compute metrics for each day, across sites and years.

In [ ]:
UASWE_metrics = run_swe_dfs[['datetime', 'swe', 'UASWE']].rename(columns={'swe':'Actual', 'UASWE':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day])


In [ ]:
run_noswe_dfs = run_cum_dfs.copy()

In [ ]:
UASWE_metrics = run_swe_dfs[['datetime', 'swe', 'UASWE']].rename(columns={'swe':'Actual', 'UASWE':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
UASWE_metrics.index.names=['Month','Day']
# UASWE_metrics.index.get_level_values('Month')
# UASWE_metrics = UASWE_metrics.drop([7,8,9], level='Month')

LM_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_LM']].rename(columns={'swe':'Actual', 'pred_swe_LM':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
LM_metrics.index.names=['Month','Day']
# LM_metrics.index.get_level_values('Month')
# LM_metrics = LM_metrics.drop([7,8,9], level='Month')

P2M_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_P2M']].rename(columns={'swe':'Actual', 'pred_swe_P2M':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
P2M_metrics.index.names=['Month','Day']
# P2M_metrics.index.get_level_values('Month')
# P2M_metrics = P2M_metrics.drop([7,8,9], level='Month')

P3M_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_P3M']].rename(columns={'swe':'Actual', 'pred_swe_P3M':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
P3M_metrics.index.names=['Month','Day']
# P3M_metrics.index.get_level_values('Month')
# P3M_metrics = P3M_metrics.drop([7,8,9], level='Month')

RF_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_RF']].rename(columns={'swe':'Actual', 'pred_swe_RF':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
RF_metrics.index.names=['Month','Day']
# RF_metrics.index.get_level_values('Month')
# RF_metrics = RF_metrics.drop([7,8,9], level='Month')

NN_metrics = run_swe_dfs[['datetime', 'swe', 'pred_swe_NN']].rename(columns={'swe':'Actual', 'pred_swe_NN':'Predicted'}).groupby([pd.to_datetime(run_swe_dfs['datetime']).dt.month, pd.to_datetime(run_swe_dfs['datetime']).dt.day]).apply(metrics)
NN_metrics.index.names=['Month','Day']
# NN_metrics.index.get_level_values('Month')
# NN_metrics = NN_metrics.drop([7,8,9], level='Month')

In [ ]:
UASWE_metrics['date'] = UASWE_metrics.index.get_level_values('Month').astype(str) + '-' + UASWE_metrics.index.get_level_values('Day').astype(str)
LM_metrics['date'] = LM_metrics.index.get_level_values('Month').astype(str) + '-' + LM_metrics.index.get_level_values('Day').astype(str)
P2M_metrics['date'] = P2M_metrics.index.get_level_values('Month').astype(str) + '-' + P2M_metrics.index.get_level_values('Day').astype(str)
P3M_metrics['date'] = P3M_metrics.index.get_level_values('Month').astype(str) + '-' + P3M_metrics.index.get_level_values('Day').astype(str)
RF_metrics['date'] = RF_metrics.index.get_level_values('Month').astype(str) + '-' + RF_metrics.index.get_level_values('Day').astype(str)
NN_metrics['date'] = NN_metrics.index.get_level_values('Month').astype(str) + '-' + NN_metrics.index.get_level_values('Day').astype(str)

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mse'], UASWE_metrics[:181]['mse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mse'], LM_metrics[:181]['mse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mse'], P2M_metrics[:181]['mse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mse'], P3M_metrics[:181]['mse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mse'], RF_metrics[:181]['mse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mse'], NN_metrics[:181]['mse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: MSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_mse_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['rmse'], UASWE_metrics[:181]['rmse']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['rmse'], LM_metrics[:181]['rmse']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['rmse'], P2M_metrics[:181]['rmse']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['rmse'], P3M_metrics[:181]['rmse']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['rmse'], RF_metrics[:181]['rmse']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['rmse'], NN_metrics[:181]['rmse']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: RMSE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
ax.set_ylim(0,200)
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_rmse_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['mae'], UASWE_metrics[:181]['mae']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['mae'], LM_metrics[:181]['mae']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['mae'], P2M_metrics[:181]['mae']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['mae'], P3M_metrics[:181]['mae']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['mae'], RF_metrics[:181]['mae']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['mae'], NN_metrics[:181]['mae']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: MAE')
ax.legend()
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_mae_climat.png', dpi=300)
plt.show()

plt.clf()
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values, pd.concat([UASWE_metrics[181:]['r2'], UASWE_metrics[:181]['r2']]).values, label='UA', color='black')
ax.plot(pd.concat([LM_metrics[181:]['date'], LM_metrics[:181]['date']]).values, pd.concat([LM_metrics[181:]['r2'], LM_metrics[:181]['r2']]).values, label='LM')
ax.plot(pd.concat([P2M_metrics[181:]['date'], P2M_metrics[:181]['date']]).values, pd.concat([P2M_metrics[181:]['r2'], P2M_metrics[:181]['r2']]).values, label='P2M')
ax.plot(pd.concat([P3M_metrics[181:]['date'], P3M_metrics[:181]['date']]).values, pd.concat([P3M_metrics[181:]['r2'], P3M_metrics[:181]['r2']]).values, label='P3M')
ax.plot(pd.concat([RF_metrics[181:]['date'], RF_metrics[:181]['date']]).values, pd.concat([RF_metrics[181:]['r2'], RF_metrics[:181]['r2']]).values, label='RF')
ax.plot(pd.concat([NN_metrics[181:]['date'], NN_metrics[:181]['date']]).values, pd.concat([NN_metrics[181:]['r2'], NN_metrics[:181]['r2']]).values, label='NN')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([UASWE_metrics[181:]['date'], UASWE_metrics[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('SWE: R2')
ax.legend()
ax.set_ylim(-1, 1)
ax.axhline(0, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/swe_eval/climatological_scheme/swe_r2_climat.png', dpi=300)
plt.show()

## Site-Year

Compute metrics by site-year.

In [ ]:
mse_swe_UASWE, mse_swe_LM, mse_swe_P2M, mse_swe_P3M, mse_swe_RF, mse_swe_NN = [], [], [], [], [], []
mae_swe_UASWE, mae_swe_LM, mae_swe_P2M, mae_swe_P3M, mae_swe_RF, mae_swe_NN = [], [], [], [], [], []
r2_swe_UASWE, r2_swe_LM, r2_swe_P2M, r2_swe_P3M, r2_swe_RF, r2_swe_NN = [], [], [], [], [], []

In [ ]:
for site in np.unique(run_swe_dfs['sitecode']):
    for wy in np.unique(run_swe_dfs['water_year']):
        site_wy_df = run_swe_dfs[(run_swe_dfs['sitecode'] == site) & (run_swe_dfs['water_year'] == wy)]
        
        if not site_wy_df.empty:
            
            mse_swe_UASWE.append(mean_squared_error(site_wy_df['swe'], site_wy_df['UASWE'], squared=True))
            mse_swe_LM.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_LM'], squared=True))
            mse_swe_P2M.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_P2M'], squared=True))
            mse_swe_P3M.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_P3M'], squared=True))
            mse_swe_RF.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_RF'], squared=True))
            mse_swe_NN.append(mean_squared_error(site_wy_df['swe'], site_wy_df['pred_swe_NN'], squared=True))     
            mae_swe_UASWE.append(np.mean(abs(site_wy_df['UASWE'] - site_wy_df['swe'])))
            mae_swe_LM.append(np.mean(abs(site_wy_df['pred_swe_LM'] - site_wy_df['swe'])))
            mae_swe_P2M.append(np.mean(abs(site_wy_df['pred_swe_P2M'] - site_wy_df['swe'])))
            mae_swe_P3M.append(np.mean(abs(site_wy_df['pred_swe_P3M'] - site_wy_df['swe'])))
            mae_swe_RF.append(np.mean(abs(site_wy_df['pred_swe_RF'] - site_wy_df['swe'])))
            mae_swe_NN.append(np.mean(abs(site_wy_df['pred_swe_NN'] - site_wy_df['swe'])))
            r2_swe_UASWE.append(r2_score(site_wy_df['swe'], site_wy_df['UASWE']))
            r2_swe_LM.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_LM']))
            r2_swe_P2M.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_P2M']))
            r2_swe_P3M.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_P3M']))
            r2_swe_RF.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_RF']))
            r2_swe_NN.append(r2_score(site_wy_df['swe'], site_wy_df['pred_swe_NN']))


In [ ]:
swe_metrics_med = {'UA SWE Reg': [np.median(mse_swe_UASWE), np.median(mae_swe_UASWE), np.median(r2_swe_UASWE)],
                   'Lin Reg': [np.median(mse_swe_LM), np.median(mae_swe_LM), np.median(r2_swe_LM)],
                   'Quad Reg': [np.median(mse_swe_P2M), np.median(mae_swe_P2M), np.median(r2_swe_P2M)],
                   'Cub Reg': [np.median(mse_swe_P3M), np.median(mae_swe_P3M), np.median(r2_swe_P3M)],
                   'RF': [np.median(mse_swe_RF), np.median(mae_swe_RF), np.median(r2_swe_RF)],
                   'NN': [np.median(mse_swe_NN), np.median(mae_swe_NN), np.median(r2_swe_NN)]
                  }
swe_metrics_mean = {'UA SWE Reg': [np.mean(mse_swe_UASWE), np.mean(mae_swe_UASWE), np.mean(r2_swe_UASWE)],
               'Lin Reg': [np.mean(mse_swe_LM), np.mean(mae_swe_LM), np.mean(r2_swe_LM)],
               'Quad Reg': [np.mean(mse_swe_P2M), np.mean(mae_swe_P2M), np.mean(r2_swe_P2M)],
               'Cub Reg': [np.mean(mse_swe_P3M), np.mean(mae_swe_P3M), np.mean(r2_swe_P3M)],
               'RF': [np.mean(mse_swe_RF), np.mean(mae_swe_RF), np.mean(r2_swe_RF)],
               'NN': [np.mean(mse_swe_NN), np.mean(mae_swe_NN), np.mean(r2_swe_NN)]
              }
swe_metrics_std = {'UA SWE Reg': [np.std(mse_swe_UASWE), np.std(mae_swe_UASWE), np.std(r2_swe_UASWE)],
               'Lin Reg': [np.std(mse_swe_LM), np.std(mae_swe_LM), np.std(r2_swe_LM)],
               'Quad Reg': [np.std(mse_swe_P2M), np.std(mae_swe_P2M), np.std(r2_swe_P2M)],
               'Cub Reg': [np.std(mse_swe_P3M), np.std(mae_swe_P3M), np.std(r2_swe_P3M)],
               'RF': [np.std(mse_swe_RF), np.std(mae_swe_RF), np.std(r2_swe_RF)],
               'NN': [np.std(mse_swe_NN), np.std(mae_swe_NN), np.std(r2_swe_NN)]
              }

In [ ]:
swe_abl_means = pd.DataFrame(swe_metrics_mean, index=['MSE','MAE','R2'])

In [ ]:
swe_abl_meds = pd.DataFrame(swe_metrics_med, index=['MSE','MAE','R2'])

In [ ]:
swe_abl_stds = pd.DataFrame(swe_metrics_std, index=['MSE','MAE','R2'])

In [ ]:
swe_abl_means.to_csv(f'{save_dir}{mode}/swe_eval/site_year_scheme/swe_abl_means.csv')
swe_abl_meds.to_csv(f'{save_dir}{mode}/swe_eval/site_year_scheme/swe_abl_meds.csv')
swe_abl_stds.to_csv(f'{save_dir}{mode}/swe_eval/site_year_scheme/swe_abl_stds.csv')

In [ ]:
swe_abl_means

In [ ]:
swe_abl_meds

# Climatological SWE Time Series

Generate climatological time series of SWE model predictions and compare it to that of SNOTEL SWE.

In [ ]:
dfs_sub = run_swe_dfs[['datetime', 'swe', 'UASWE', 'pred_swe_UASWE',
       'pred_swe_LM', 'pred_swe_P2M', 'pred_swe_P3M', 'pred_swe_RF',
       'pred_swe_NN']]

In [ ]:
snotel_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['swe'].mean()
uaswe_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['UASWE'].mean()
myswe_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_UASWE'].mean()
lm_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_LM'].mean()
p2m_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_P2M'].mean()
p3m_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_P3M'].mean()
rf_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_RF'].mean()
nn_swe_mean = dfs_sub.groupby([(pd.to_datetime(dfs_sub['datetime']).dt.month),(pd.to_datetime(dfs_sub['datetime']).dt.day)])['pred_swe_NN'].mean()

In [ ]:
# print(lm_swe_mean, p2m_swe_mean, p3m_swe_mean, rf_swe_mean)

In [ ]:
snotel_swe_mean.index.names = ['Month', 'Day']
uaswe_swe_mean.index.names = ['Month', 'Day']
myswe_swe_mean.index.names = ['Month', 'Day']
lm_swe_mean.index.names = ['Month', 'Day']
p2m_swe_mean.index.names = ['Month', 'Day']
p3m_swe_mean.index.names = ['Month', 'Day']
rf_swe_mean.index.names = ['Month', 'Day']
nn_swe_mean.index.names = ['Month', 'Day']

In [ ]:
snotel_swe_mean = snotel_swe_mean.to_frame()
uaswe_swe_mean = uaswe_swe_mean.to_frame()
myswe_swe_mean = myswe_swe_mean.to_frame()
lm_swe_mean = lm_swe_mean.to_frame()
p2m_swe_mean = p2m_swe_mean.to_frame()
p3m_swe_mean = p3m_swe_mean.to_frame()
rf_swe_mean = rf_swe_mean.to_frame()
nn_swe_mean = nn_swe_mean.to_frame()

In [ ]:
snotel_swe_mean['date'] = snotel_swe_mean.index.get_level_values('Month').astype(str) + '-' + snotel_swe_mean.index.get_level_values('Day').astype(str)
uaswe_swe_mean['date'] = uaswe_swe_mean.index.get_level_values('Month').astype(str) + '-' + uaswe_swe_mean.index.get_level_values('Day').astype(str)
myswe_swe_mean['date'] = myswe_swe_mean.index.get_level_values('Month').astype(str) + '-' + myswe_swe_mean.index.get_level_values('Day').astype(str)
lm_swe_mean['date'] = lm_swe_mean.index.get_level_values('Month').astype(str) + '-' + lm_swe_mean.index.get_level_values('Day').astype(str)
p2m_swe_mean['date'] = p2m_swe_mean.index.get_level_values('Month').astype(str) + '-' + p2m_swe_mean.index.get_level_values('Day').astype(str)
p3m_swe_mean['date'] = p3m_swe_mean.index.get_level_values('Month').astype(str) + '-' + p3m_swe_mean.index.get_level_values('Day').astype(str)
rf_swe_mean['date'] = rf_swe_mean.index.get_level_values('Month').astype(str) + '-' + rf_swe_mean.index.get_level_values('Day').astype(str)
nn_swe_mean['date'] = nn_swe_mean.index.get_level_values('Month').astype(str) + '-' + nn_swe_mean.index.get_level_values('Day').astype(str)

In [ ]:
fig, ax = plt.subplots(figsize=(24, 6))
ax.plot(pd.concat([snotel_swe_mean[181:]['date'], snotel_swe_mean[:181]['date']]).values, pd.concat([snotel_swe_mean[181:]['swe'], snotel_swe_mean[:181]['swe']]).values, label='SNOTEL SWE', color='black')
ax.plot(pd.concat([uaswe_swe_mean[181:]['date'], uaswe_swe_mean[:181]['date']]).values, pd.concat([uaswe_swe_mean[181:]['UASWE'], uaswe_swe_mean[:181]['UASWE']]).values, label='UA SWE', color='grey')
ax.plot(pd.concat([myswe_swe_mean[181:]['date'], myswe_swe_mean[:181]['date']]).values, pd.concat([myswe_swe_mean[181:]['pred_swe_UASWE'], myswe_swe_mean[:181]['pred_swe_UASWE']]).values, label='UA SWE [MY]')
ax.plot(pd.concat([lm_swe_mean[181:]['date'], lm_swe_mean[:181]['date']]).values, pd.concat([lm_swe_mean[181:]['pred_swe_LM'], lm_swe_mean[:181]['pred_swe_LM']]).values, label='LM-derived SWE')
ax.plot(pd.concat([p2m_swe_mean[181:]['date'], p2m_swe_mean[:181]['date']]).values, pd.concat([p2m_swe_mean[181:]['pred_swe_P2M'], p2m_swe_mean[:181]['pred_swe_P2M']]).values, label='P2M-derived SWE')
ax.plot(pd.concat([p3m_swe_mean[181:]['date'], p3m_swe_mean[:181]['date']]).values, pd.concat([p3m_swe_mean[181:]['pred_swe_P3M'], p3m_swe_mean[:181]['pred_swe_P3M']]).values, label='P3M-derived SWE')
ax.plot(pd.concat([rf_swe_mean[181:]['date'], rf_swe_mean[:181]['date']]).values, pd.concat([rf_swe_mean[181:]['pred_swe_RF'], rf_swe_mean[:181]['pred_swe_RF']]).values, label='RF-derived SWE')
ax.plot(pd.concat([nn_swe_mean[181:]['date'], nn_swe_mean[:181]['date']]).values, pd.concat([nn_swe_mean[181:]['pred_swe_NN'], nn_swe_mean[:181]['pred_swe_NN']]).values, label='NN-derived SWE')
ax.set_xticks(ax.get_xticks()[::30])   # tick at every 30th entry
ax.set_xticklabels(ax.get_xticks(), rotation=90)
ax.set_xticklabels(list(pd.concat([snotel_swe_mean[181:]['date'], snotel_swe_mean[:181]['date']]).values)[::30])    # tick labels at every 30th entry
ax.set_title('Mean SWE on Each Date Across Sites and Years')
ax.set_ylabel('Mean SWE (mm)')
ax.set_xlabel('Date')
ax.legend()
# ax.set_ylim(0,700)
# ax.axhline(200, linestyle='--', color='lightgrey')
# plt.savefig(f'{save_dir}{mode}/mean_swe_ts.png', dpi=300)